In [1]:
import pandas as pd

In [2]:
df_vendas = pd.read_csv('../../data/raw/tb_vendas.csv')
df_vendas['dt_venda'] = pd.to_datetime(df_vendas['dt_venda'])
df_vendas.rename(columns={"cod_material": "cod_produto"}, inplace=True)
df_vendas.head()

,dt_venda,cod_pedido,canal,des_estado_franquia,key_loja,cod_produto,categoria_produto,sub_categoria_produto,linha_produto,vlr_venda_tabelado,vlr_desconto
0,2024-05-10,386761455,FISCO,SP,1,1428,217.0,422944,731659,248.7,0.00
1,2024-03-25,377623716,DIGITAL,SP,1,1428,217.0,422944,731659,248.7,88.23
2,2024-03-19,376372149,DIGITAL,SP,1,1428,217.0,422944,731659,248.7,88.23
3,2024-10-23,414882456,DIGITAL,SP,1,1428,217.0,422944,731659,248.7,0.00
4,2024-02-27,371457908,DIGITAL,SP,1,1428,217.0,422944,731659,230.7,0.00


In [3]:
df_estoque_venda_interp = pd.read_csv('../../data/processed/dados_estoque_vendas_interpolado.csv')
df_estoque_venda_interp['dt_estoque'] = pd.to_datetime(df_estoque_venda_interp['dt_estoque'])
df_estoque_venda_interp = df_estoque_venda_interp.sort_values(['key_loja', 'cod_produto', 'dt_estoque'])
df_estoque_venda_interp

C:\Users\Carlos\AppData\Local\Temp\ipykernel_5372\2034582778.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estoque_venda_interp = pd.read_csv('../../data/processed/dados_estoque_vendas_interpolado.csv')


,key_loja,cod_produto,dt_estoque,qt_estoque,is_ruptura,qtd_vendas,tipo_demanda,qtd_venda_interp
120822,1,1428,2024-01-01,108.0,0,0.0,NaN,0.0
120823,1,1428,2024-01-02,108.0,0,0.0,NaN,0.0
120824,1,1428,2024-01-03,108.0,0,0.0,NaN,0.0
120825,1,1428,2024-01-04,108.0,0,0.0,NaN,0.0
120826,1,1428,2024-01-05,108.0,0,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...
51825,8,84206,2024-12-27,0.0,1,NaN,Smooth,1.0
51826,8,84206,2024-12-28,0.0,1,NaN,Smooth,1.0
51827,8,84206,2024-12-29,0.0,1,NaN,Smooth,1.0
51828,8,84206,2024-12-30,0.0,1,NaN,Smooth,1.0


# Features Info Produto

In [4]:
df_descr_loja_prod = df_vendas[['des_estado_franquia', 'key_loja', 'cod_produto', 'categoria_produto', 'sub_categoria_produto', 'linha_produto']]
df_descr_loja_prod = df_descr_loja_prod.groupby(['key_loja', 'cod_produto'], as_index=False).agg({'des_estado_franquia':'first',
                                                                                                  'categoria_produto':'first',
                                                                                                  'sub_categoria_produto':'first',
                                                                                                  'linha_produto':'first'})
df_descr_loja_prod.head()

,key_loja,cod_produto,des_estado_franquia,categoria_produto,sub_categoria_produto,linha_produto
0,1,1428,SP,217.0,422944,731659
1,1,1640,SP,217.0,422944,731659
2,1,1650,SP,217.0,422944,731659
3,1,22230,SP,157293.0,483679,85436
4,1,25458,SP,157293.0,483679,284036


In [5]:
df_feat_1 = df_estoque_venda_interp.merge(df_descr_loja_prod, on=['key_loja', 'cod_produto'], how='left')
df_feat_1.head()

,key_loja,cod_produto,dt_estoque,qt_estoque,is_ruptura,qtd_vendas,tipo_demanda,qtd_venda_interp,des_estado_franquia,categoria_produto,sub_categoria_produto,linha_produto
0,1,1428,2024-01-01,108.0,0,0.0,NaN,0.0,SP,217.0,422944,731659
1,1,1428,2024-01-02,108.0,0,0.0,NaN,0.0,SP,217.0,422944,731659
2,1,1428,2024-01-03,108.0,0,0.0,NaN,0.0,SP,217.0,422944,731659
3,1,1428,2024-01-04,108.0,0,0.0,NaN,0.0,SP,217.0,422944,731659
4,1,1428,2024-01-05,108.0,0,2.0,NaN,2.0,SP,217.0,422944,731659


In [6]:
df_vendas_agrupado = (
    df_vendas.groupby(['dt_venda', 'key_loja', 'cod_produto'], as_index=False)
     .agg(qtd_vendas=('vlr_venda_tabelado', 'count'),
     vlr_venda_tabelado_mean=('vlr_venda_tabelado', 'mean')))

df_vendas_agrupado.head()

,dt_venda,key_loja,cod_produto,qtd_vendas,vlr_venda_tabelado_mean
0,2024-01-02,1,48062,1,284.70
1,2024-01-02,1,48635,1,419.70
2,2024-01-02,1,82060,1,1017.00
3,2024-01-02,2,1650,1,167.70
4,2024-01-02,2,48060,2,584.55


In [7]:
df_feat_1 = df_feat_1.sort_values(['dt_estoque', 'key_loja', 'cod_produto']).reset_index(drop=True)
df_vendas_agrupado = df_vendas_agrupado.sort_values(['dt_venda', 'key_loja', 'cod_produto']).reset_index(drop=True)

df_feat_1 = pd.merge_asof(
    df_feat_1,
    df_vendas_agrupado,
    by=['key_loja', 'cod_produto'],
    left_on='dt_estoque',
    right_on='dt_venda',
    direction='backward'
)
df_feat_1.drop(columns=['qtd_vendas_y', 'dt_venda'], inplace=True)
df_feat_1.rename(columns={'qtd_vendas_x':'qtd_vendas'},inplace=True)
df_feat_1.head()

,key_loja,cod_produto,dt_estoque,qt_estoque,is_ruptura,qtd_vendas,tipo_demanda,qtd_venda_interp,des_estado_franquia,categoria_produto,sub_categoria_produto,linha_produto,vlr_venda_tabelado_mean
0,1,1428,2024-01-01,108.0,0,0.0,NaN,0.0,SP,217.0,422944,731659,NaN
1,1,1640,2024-01-01,83.0,0,0.0,NaN,0.0,SP,217.0,422944,731659,NaN
2,1,1650,2024-01-01,102.0,0,0.0,NaN,0.0,SP,217.0,422944,731659,NaN
3,1,22230,2024-01-01,0.0,1,NaN,NaN,1.0,SP,157293.0,483679,85436,NaN
4,1,25458,2024-01-01,46.0,0,0.0,Smooth,0.0,SP,157293.0,483679,284036,NaN


# Features Estatisticas

In [8]:
def create_rolling_statistics(df, col_base, janelas):
    for janela in janelas:
        df[f'{col_base}_media_{janela}d'] = (
            df.groupby(['key_loja', 'cod_produto'])[col_base]
              .transform(lambda x: x.shift(1).rolling(janela).mean())
        )
        df[f'{col_base}_mediana_{janela}d'] = (
            df.groupby(['key_loja', 'cod_produto'])[col_base]
              .transform(lambda x: x.shift(1).rolling(janela).median())
        )
        df[f'{col_base}_max_{janela}d'] = (
            df.groupby(['key_loja', 'cod_produto'])[col_base]
              .transform(lambda x: x.shift(1).rolling(janela).max())
        )
        df[f'{col_base}_min_{janela}d'] = (
            df.groupby(['key_loja', 'cod_produto'])[col_base]
              .transform(lambda x: x.shift(1).rolling(janela).min())
        )

    return df

def gerar_ratios(df, prefixo, col_base, janelas):
    for i in range(len(janelas)):
        for j in range(i+1, len(janelas)):
            j1 = janelas[i]
            j2 = janelas[j]
            col1 = f'{col_base}_{prefixo}_{j1}d'
            col2 = f'{col_base}_{prefixo}_{j2}d'
            df[f'{prefixo}_{j1}d_{j2}d_ratio_{col_base}'] = df[col1] / df[col2]
    return df

janelas = [1, 7, 30, 60]

# Estatistica sobre as Vendas ===========
df_feat_2 = create_rolling_statistics(df_feat_1.copy(), 'qtd_vendas', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'media', 'qtd_vendas', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'mediana', 'qtd_vendas', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'max', 'qtd_vendas', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'min', 'qtd_vendas', janelas)

# Estatistica sobre as Valor do Produto ===========
df_feat_2 = create_rolling_statistics(df_feat_2.copy(), 'vlr_venda_tabelado_mean', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'media', 'vlr_venda_tabelado_mean', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'mediana', 'vlr_venda_tabelado_mean', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'max', 'vlr_venda_tabelado_mean', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'min', 'vlr_venda_tabelado_mean', janelas)

# Estatistica sobre as Qtd Estoque ===========
df_feat_2 = create_rolling_statistics(df_feat_2.copy(), 'qt_estoque', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'media', 'qt_estoque', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'mediana', 'qt_estoque', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'max', 'qt_estoque', janelas)
df_feat_2 = gerar_ratios(df_feat_2, 'min', 'qt_estoque', janelas)

df_feat_2.head()

,key_loja,cod_produto,dt_estoque,qt_estoque,is_ruptura,qtd_vendas,tipo_demanda,qtd_venda_interp,des_estado_franquia,categoria_produto,...,max_1d_60d_ratio_qt_estoque,max_7d_30d_ratio_qt_estoque,max_7d_60d_ratio_qt_estoque,max_30d_60d_ratio_qt_estoque,min_1d_7d_ratio_qt_estoque,min_1d_30d_ratio_qt_estoque,min_1d_60d_ratio_qt_estoque,min_7d_30d_ratio_qt_estoque,min_7d_60d_ratio_qt_estoque,min_30d_60d_ratio_qt_estoque
0,1,1428,2024-01-01,108.0,0,0.0,NaN,0.0,SP,217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1640,2024-01-01,83.0,0,0.0,NaN,0.0,SP,217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1650,2024-01-01,102.0,0,0.0,NaN,0.0,SP,217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,22230,2024-01-01,0.0,1,NaN,NaN,1.0,SP,157293.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,25458,2024-01-01,46.0,0,0.0,Smooth,0.0,SP,157293.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Features Temporal

In [9]:
df_feat_2['day_of_the_week'] = df_feat_2['dt_estoque'].dt.dayofweek
df_feat_2

,key_loja,cod_produto,dt_estoque,qt_estoque,is_ruptura,qtd_vendas,tipo_demanda,qtd_venda_interp,des_estado_franquia,categoria_produto,...,max_7d_30d_ratio_qt_estoque,max_7d_60d_ratio_qt_estoque,max_30d_60d_ratio_qt_estoque,min_1d_7d_ratio_qt_estoque,min_1d_30d_ratio_qt_estoque,min_1d_60d_ratio_qt_estoque,min_7d_30d_ratio_qt_estoque,min_7d_60d_ratio_qt_estoque,min_30d_60d_ratio_qt_estoque,day_of_the_week
0,1,1428,2024-01-01,108.0,0,0.0,NaN,0.0,SP,217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,1640,2024-01-01,83.0,0,0.0,NaN,0.0,SP,217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,1650,2024-01-01,102.0,0,0.0,NaN,0.0,SP,217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,22230,2024-01-01,0.0,1,NaN,NaN,1.0,SP,157293.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,25458,2024-01-01,46.0,0,0.0,Smooth,0.0,SP,157293.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246872,8,83772,2024-12-31,0.0,1,NaN,Lumpy,NaN,MG,157293.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
246873,8,83845,2024-12-31,27.0,0,0.0,Smooth,0.0,MG,157293.0,...,1.0,1.0,1.0,27.0,inf,inf,inf,inf,NaN,1
246874,8,84203,2024-12-31,0.0,1,NaN,Smooth,1.0,MG,217.0,...,0.0,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,1
246875,8,84204,2024-12-31,0.0,1,NaN,Lumpy,NaN,MG,217.0,...,1.0,1.0,1.0,inf,inf,inf,NaN,NaN,NaN,1


# Separando os Dados

- Vou separar os dados por tipo de demanda

In [10]:
df_feat_2['tipo_demanda'].value_counts()

tipo_demanda
Smooth          51830
Lumpy           38922
Erratic         28606
Intermittent     1464
Name: count, dtype: int64

In [11]:
df = df_feat_2[df_feat_2['tipo_demanda']=='Smooth']
df.to_csv('../../data/processed/dados_features_Smooth.csv', index_label=False)

df = df_feat_2[df_feat_2['tipo_demanda']=='Lumpy']
df.to_csv('../../data/processed/dados_features_Lumpy.csv', index_label=False)

df = df_feat_2[df_feat_2['tipo_demanda']=='Erratic']
df.to_csv('../../data/processed/dados_features_Erratic.csv', index_label=False)

df = df_feat_2[df_feat_2['tipo_demanda']=='Intermittent']
df.to_csv('../../data/processed/dados_features_Intermittent.csv', index_label=False)

df = df_feat_2[df_feat_2['tipo_demanda'].isnull()]
df.to_csv('../../data/processed/dados_features_Sem_Tipo.csv', index_label=False)